In [1]:
#import the necessary packages
import numpy as np
import pandas as pd

### Read in EAE document
Read in EAE Data

In [2]:
# Read in the application data scrapped from EAE pdf
df_merge = pd.read_csv('final_merged_predict.csv', index_col=0)
df_merge.head()

,UID,Choice,Course Code,Final Writeup,predict_recommend
0,C35A1W1,1,C35,I believe in this technological advanced world...,0
1,C35A2W1,1,C35,The Business and Financial Technology course h...,0
2,C35A3W1,1,C35,I fancy numbers and I always want to know how ...,0
3,C35A4W1,1,C35,I foresee that there is a growth and demand in...,1
4,C35A5W1,1,C35,We are into high technology era where people u...,0


### Load LDA Model and Score data
Load finalize LDA model and score the EAE data.

In [3]:
import gensim 
from sklearn.feature_extraction.text import CountVectorizer

ldamodel = gensim.models.LdaModel.load('model20try.gensim', mmap='r')

# Use CountVectorizor to find three letter tokens, remove stop_words, 
# remove tokens that don't appear in at least 20 documents,
# remove tokens that appear in more than 20% of the documents
vect = CountVectorizer(min_df=20, max_df=0.2, stop_words='english', 
                       token_pattern='(?u)\\b\\w\\w\\w+\\b')
# Fit and transform
X = vect.fit_transform(df_merge['Final Writeup'])

In [4]:
def topic_prediction(my_document):
    string_input = [my_document]
    X = vect.transform(string_input)

    # Convert sparse matrix to gensim corpus.
    corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)
    output = list(ldamodel[corpus])[0]
    topics = sorted(output,key=lambda x:x[1],reverse=True)
    return topics[0]

In [5]:
predictedtopic = []

for ind in df_merge.index:
    predictedtopic.append(list(topic_prediction(df_merge['Final Writeup'][ind])))

print(predictedtopic[2])

[14, 0.48917955]


In [6]:
predictionDF = pd.DataFrame(predictedtopic, columns=['predictedTopic', 'predProb'])
predictionDF.head()

,predictedTopic,predProb
0,9,0.397608
1,4,0.314243
2,14,0.489180
3,17,0.986028
4,10,0.984165


In [7]:
final = pd.merge(df_merge, predictionDF, left_index=True, right_index=True)
final.head()

,UID,Choice,Course Code,Final Writeup,predict_recommend,predictedTopic,predProb
0,C35A1W1,1,C35,I believe in this technological advanced world...,0,9,0.397608
1,C35A2W1,1,C35,The Business and Financial Technology course h...,0,4,0.314243
2,C35A3W1,1,C35,I fancy numbers and I always want to know how ...,0,14,0.489180
3,C35A4W1,1,C35,I foresee that there is a growth and demand in...,1,17,0.986028
4,C35A5W1,1,C35,We are into high technology era where people u...,0,10,0.984165


In [8]:
final.to_csv('finalfinal.csv')